### A) Data Preparation

In [2]:
import pandas as pd
path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/ML based speckle POC/"
df = pd.read_csv(path+"SNR_R5_ww51.4.csv")

In [2]:
df.shape

(53772, 1833)

#### Check and remove rows with blank delta

In [3]:
#check for rows with blank delta
import numpy as np
df1 = df[df["DELTA"].isnull()]
df1

,VID,ULT@MIDAS_6261_U1,SORTLOT,SORTLOT7,WAFER,XLOC,YLOC,ULT@MIDAS_6261_U2,IB@6261[CLASSHOT],FB@6261[CLASSHOT],...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TX_EN_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TXD0_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TXD1_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_RMII_CLK_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_CLK_RX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_RX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_TX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_WAKE_N_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_XTAL_VALID_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_PMSYNC_CPU
22481,M1XH703100536,N0366610_230_0_6,N0366610,N036661,230,0,6,H0311010_638_-6_-1,45,4543,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22485,M1XH703100244,N0342520_276_-3_0,N0342520,N034252,276,-3,0,H0311010_638_-11_-10,27,2755,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22577,M1XH703100936,N0423170_659_-3_3,N0423170,N042317,659,-3,3,H0403340_372_2_13,21,2155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22585,M1BH477201065,N0423170_700_3_-9,N0423170,N042317,700,3,-9,H0271700_785_14_1,9,997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22624,M1XH703101227,N0422350_116_-3_-3,N0422350,N042235,116,-3,-3,H0403340_372_-4_-4,60,6005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53524,M1MM581500781,N0423170_669_4_-2,N0423170,N042317,669,4,-2,H0311010_643_-6_-18,21,2150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53543,M1MM581500722,N0446570_120_-4_6,N0446570,N044657,120,-4,6,H0311010_643_-5_2,61,6180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53557,M1MM581500744,N0411290_435_4_4,N0411290,N041129,435,4,4,H0311010_567_5_18,45,4550,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53652,M1JJ817501248,N0422200_050_6_2,N0422200,N042220,50,6,2,H0311010_568_1_-18,21,2148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#make folder for dataprep if it doesn't exist
import os
dataprep_path = path + 'DataPreparation/SNR_R5_ww51.4/' 
if not os.path.exists(dataprep_path):
    os.makedirs(dataprep_path)
    print("DataPreparation folder created!")

In [6]:
df1.to_csv(dataprep_path+"Blank_delta.csv",index=False)

In [7]:
#remove rows with blank delta
import numpy as np
df = df[df["DELTA"].notnull()]
df

,VID,ULT@MIDAS_6261_U1,SORTLOT,SORTLOT7,WAFER,XLOC,YLOC,ULT@MIDAS_6261_U2,IB@6261[CLASSHOT],FB@6261[CLASSHOT],...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TX_EN_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TXD0_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TXD1_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_RMII_CLK_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_CLK_RX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_RX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_TX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_WAKE_N_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_XTAL_VALID_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_PMSYNC_CPU
0,M00C045U00024,N0284110_927_0_4,N0284110,N028411,927,0,4,H0206320_095_4_-4,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M00C045U00050,N0292750_182_3_-6,N0292750,N029275,182,3,-6,H0206290_057_-5_-17,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M00C045U00123,N0292750_221_-5_-5,N0292750,N029275,221,-5,-5,H0206290_057_0_-14,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M00C045U00134,N0284110_927_-3_-8,N0284110,N028411,927,-3,-8,H0206320_095_4_-6,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M00C045U00135,N0284110_927_4_-6,N0284110,N028411,927,4,-6,H0206320_095_4_-7,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53767,M1JJ817501153,N0422200_050_5_-2,N0422200,N042220,50,5,-2,H0311010_568_4_14,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53768,M1JJ817501233,N0422200_050_-3_1,N0422200,N042220,50,-3,1,H0311010_568_4_-11,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53769,M1JJ817501241,N0422200_050_0_4,N0422200,N042220,50,0,4,H0311010_568_4_-8,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53770,M1JJ817501242,N0422200_050_0_5,N0422200,N042220,50,0,5,H0311010_568_4_-9,1,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df["DELTA"].value_counts()

0.0    46342
1.0     6432
2.0      130
Name: DELTA, dtype: int64

In [9]:
#create new column SPECKLE as target
import numpy as np
df["SPECKLE"] = np.where(df["DELTA"]==0,0,1)
cols = df.columns.tolist()
cols = [cols[-1]]+cols[:-1] #move speckle col to the front
df = df.reindex(columns=cols)

In [10]:
df["SPECKLE"].value_counts()

0    46342
1     6562
Name: SPECKLE, dtype: int64

In [ ]:
df.head()

In [ ]:
# pd.options.display.max_seq_items = 2000
# df.columns

In [11]:
df = df.reset_index(drop=True)
df.shape

(52904, 1834)

In [ ]:
# df.describe()

In [12]:
df.to_csv(dataprep_path+"df_Blankdeltarem.csv",index=False)

### 1) Check for VID duplicates (Unique VID)

In [13]:
df.shape

(52904, 1834)

In [14]:
df["SPECKLE"].value_counts()

0    46342
1     6562
Name: SPECKLE, dtype: int64

In [15]:
def check_duplicates(df):
    """
    Check for duplicates in VID 
    df[DataFrame]: input df    
    """
    if df['VID'].duplicated().any():
        print("There are duplicates in VID")
        df_dup = df[df.duplicated(subset=['VID'])] #get the duplicates in df, returns repeated rows only 
        df= df[df["VID"].isin(df_dup["VID"])] #get all the duplicates in df, returns original + repeated rows 
        return df
    else:
        print("There are no duplicates in VID")
    

In [16]:
df_dup = check_duplicates(df)
df_dup #78 rows of duplicates, 39 duplicated VIDs -> ONLY 1 VID has values, the rest all blanks

There are duplicates in VID


,SPECKLE,VID,ULT@MIDAS_6261_U1,SORTLOT,SORTLOT7,WAFER,XLOC,YLOC,ULT@MIDAS_6261_U2,IB@6261[CLASSHOT],...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TX_EN_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TXD0_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TXD1_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_RMII_CLK_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_CLK_RX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_RX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_TX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_WAKE_N_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_XTAL_VALID_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_PMSYNC_CPU
10018,1,M13N2V2900237,N0395200_215_3_1,N0395200,N039520,215,3,1,H0271700_766_1_12,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10061,1,M13N2V2900438,N0410300_364_-4_2,N0410300,N041030,364,-4,2,H0280420_608_-1_-3,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10524,1,M14M6U9601241,N0371260_365_4_-2,N0371260,N037126,365,4,-2,H0403360_811_-8_-1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11372,1,M165W4T400428,N0371240_404_3_-7,N0371240,N037124,404,3,-7,H0403360_811_-3_-12,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11373,1,M165W4T400910,N0395050_466_1_4,N0395050,N039505,466,1,4,H0402280_031_-2_15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48381,1,M1VP888001021,N0382520_389_-2_-3,N0382520,N038252,389,-2,-3,H0403360_865_1_-22,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51467,1,M1N96V6800528,N0382620_355_2_7,N0382620,N038262,355,2,7,H0271700_768_-2_-11,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51524,1,M1S523K900060,N0394880_119_-6_-2,N0394880,N039488,119,-6,-2,H0271700_768_15_2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51748,1,M1S523K900841,N0394900_180_-1_-1,N0394900,N039490,180,-1,-1,H0311010_648_-7_-4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_dup.to_csv(dataprep_path+"DuplicateVID.csv",index=False)

In [18]:
#only duplicate VID M1BH477200042 has values, the rest all NAN
df_dup_withval = df.loc[df['VID'] == 'M1BH477200042']
df_dup_withval

,SPECKLE,VID,ULT@MIDAS_6261_U1,SORTLOT,SORTLOT7,WAFER,XLOC,YLOC,ULT@MIDAS_6261_U2,IB@6261[CLASSHOT],...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TX_EN_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TXD0_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TXD1_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_RMII_CLK_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_CLK_RX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_RX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_TX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_WAKE_N_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_XTAL_VALID_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_PMSYNC_CPU
13581,1,M1BH477200042,N0366610_224_-2_-5,N0366610,N036661,224,-2,-5,H0311010_638_-7_-21,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23429,1,M1BH477200042,N0366610_224_-2_-5,N0366610,N036661,224,-2,-5,H0311010_638_-7_-21,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
def duplicates_handling(df,keep):
    """
    Remove duplicates if there is any
    df[DataFrame]: input df
    keep: {‘first’, ‘last’, False}: Determines which duplicates (if any) to keep. 
    - first : Drop duplicates except for the first occurrence. 
    - last : Drop duplicates except for the last occurrence. 
    - False : Drop all duplicates.
    """
    df = df.drop_duplicates(subset=['VID'],keep=keep) #remove duplicates from df
    return df

In [20]:
#drop all duplicates for 38 VIDs, take unique row for VID M1BH477200042
df = duplicates_handling(df,keep=False)
df = pd.concat([df, df_dup_withval], ignore_index=True)
df = duplicates_handling(df,keep="first")
df

,SPECKLE,VID,ULT@MIDAS_6261_U1,SORTLOT,SORTLOT7,WAFER,XLOC,YLOC,ULT@MIDAS_6261_U2,IB@6261[CLASSHOT],...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TX_EN_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TXD0_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_NCSI_TXD1_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_RMII_CLK_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_CLK_RX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_RX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_SBLINK_TX_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_WAKE_N_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_NAC_XTAL_VALID_CPU,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_FC1@132110_PMSYNC_CPU
0,1,M00C045U00024,N0284110_927_0_4,N0284110,N028411,927,0,4,H0206320_095_4_-4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,M00C045U00050,N0292750_182_3_-6,N0292750,N029275,182,3,-6,H0206290_057_-5_-17,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,M00C045U00123,N0292750_221_-5_-5,N0292750,N029275,221,-5,-5,H0206290_057_0_-14,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,M00C045U00134,N0284110_927_-3_-8,N0284110,N028411,927,-3,-8,H0206320_095_4_-6,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,M00C045U00135,N0284110_927_4_-6,N0284110,N028411,927,4,-6,H0206320_095_4_-7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52822,0,M1JJ817501233,N0422200_050_-3_1,N0422200,N042220,50,-3,1,H0311010_568_4_-11,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52823,0,M1JJ817501241,N0422200_050_0_4,N0422200,N042220,50,0,4,H0311010_568_4_-8,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52824,0,M1JJ817501242,N0422200_050_0_5,N0422200,N042220,50,0,5,H0311010_568_4_-9,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52825,0,M1JJ817501234,N0422200_050_-3_0,N0422200,N042220,50,-3,0,H0311010_568_4_-12,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
check_duplicates(df)

There are no duplicates in VID


In [22]:
df["SPECKLE"].value_counts()

0    46342
1     6485
Name: SPECKLE, dtype: int64

In [23]:
df.to_csv(dataprep_path+"df_DuplicatesRem.csv",index=False)

### 2 ) NA handling (Part 1 - Remove rows/columns with 100% NA)

In [25]:
# check column for nulls
def check_colna(df):
    """
    Check each column for nulls. Returns Feature, total null and % of null for each column
    df[DataFrame]: dataframe
    
    """
    colna_df = pd.DataFrame(columns =["Feature", "Total Null", "% of Null"])
    for col in df.columns: 
        #checking if there is any null in the column

        if df[col].isnull().sum()>0: 
            
            # if null present, total number of null in the column stores here
            total_null = df[col].isnull().sum() 
            new_row = {'Feature':col, 'Total Null':total_null, '% of Null':total_null*100/len(df)}
            #append row to the dataframe
            colna_df = colna_df.append(new_row, ignore_index=True)
            
    colna_df= colna_df.sort_values("% of Null", ascending=False)    
    return colna_df  

# check rows for nulls
def check_rowna(df,supporting_fs):  
    """
    Check each row for nulls.Returns VID, total null and % of null for each row
    df[DataFrame]: dataframe
    supporting_fs[list]: all features not used for ML except VID and SPECKLE(target)
    """
    df = df.drop(supporting_fs,axis=1)    
    colrow_df = pd.DataFrame(columns =["SPECKLE","VID", "Total Null", "% of Null"])
    for i in df.index: 
        #checking if there is any null in the row
        if df.iloc[i].isnull().sum()>0:             
            # if null present, total number of null in the row stores here
            total_null = df.iloc[i].isnull().sum() 
            new_row = {'SPECKLE':df.iloc[i,0],'VID':df.iloc[i,1], 'Total Null':total_null, '% of Null':round(total_null*100/(len(df.columns)-2),2)}
            #append row to the dataframe
            colrow_df = colrow_df.append(new_row, ignore_index=True)
            
    colrow_df= colrow_df.sort_values("% of Null", ascending=False)    
    return colrow_df   

# Drop columns based on NA threshold limit
def drop_NAcol(df,NA_limit):
    '''
    Drops columns based on proportion of NA in column
    df[DataFrame]: df
    NA_limit[float/int]: Columns with proportion of NA above NA_limit will be dropped
    '''
    threshold = len(df)*(1-NA_limit)
    df=df.dropna(axis=1, thresh=threshold)
    print(df.shape)
    return df


#### a) Check column for 100% nulls and remove them

In [26]:
print(df.shape)
print(df["SPECKLE"].value_counts())

(52827, 1834)
0    46342
1     6485
Name: SPECKLE, dtype: int64


#### i) check 100% nulls in whole dataset and remove them

In [27]:
#check column na for whole df
colna_df =check_colna(df)
colna_df

,Feature,Total Null,% of Null
1812,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.000000
1308,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.000000
1316,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.000000
1315,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.000000
1314,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.000000
...,...,...,...
1497,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731
1496,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731
1495,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731
1494,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731


In [28]:
#check columns with 100% NA
colna_df.loc[colna_df["% of Null"] ==100]

,Feature,Total Null,% of Null
1812,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.0
1308,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.0
1316,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.0
1315,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.0
1314,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.0
...,...,...,...
1760,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.0
1756,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.0
1757,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.0
1758,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_F...,52827,100.0


In [29]:
print(len(colna_df.loc[colna_df["% of Null"] ==100]))
colna_df.to_csv(dataprep_path+"NA_Cols.csv",index=False)

174


In [30]:
#drop columns with all NA
df = df.dropna(axis=1, how='all')
df.shape

(52827, 1660)

In [31]:
check_colna(df)

,Feature,Total Null,% of Null
1024,TPI_VCC::CONT_VCCVNN_HC_E_ETEND_X_X_X_X_POSTSU...,52825,99.996214
997,TPI_VCC::CONT_VCCDDRDQ_HC_K_ETEND_X_X_X_X_SNR_...,52825,99.996214
1025,TPI_VCC::CONT_VCCVNN_HC_E_ETEND_X_X_X_X_SNR_MI...,52825,99.996214
967,TPI_VCC::CONT_VCCCLM_HC_K_ETEND_X_X_X_X_SNR_OL...,52825,99.996214
970,TPI_VCC::CONT_VCCCOREM01_HC_E_ETEND_X_X_X_X_PO...,52825,99.996214
...,...,...,...
1460,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731
1459,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731
1458,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731
1457,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731


#### ii) check 100% nulls in column for speckle only and remove them

In [32]:
df.shape

(52827, 1660)

In [33]:
#check column na for speckle
colna_speckle_df =check_colna(df.loc[df['SPECKLE'] == 1])
colna_speckle_df

,Feature,Total Null,% of Null
1007,TPI_VCC::CONT_VCCINF_HC_E_ETEND_X_X_X_X_SNR_MI...,6485,100.000000
961,TPI_VCC::CONT_VCCCLM_HC_E_ETEND_X_X_X_X_POSTSU...,6485,100.000000
970,TPI_VCC::CONT_VCCCOREM01_HC_E_ETEND_X_X_X_X_PO...,6485,100.000000
971,TPI_VCC::CONT_VCCCOREM01_HC_E_ETEND_X_X_X_X_SN...,6485,100.000000
1066,TPI_VCC::CONT_X_SCREEN_K_ETEND_X_X_X_X_SNR_ADT...,6485,100.000000
...,...,...,...
1458,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4694,72.382421
1457,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4694,72.382421
1456,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4694,72.382421
1455,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4694,72.382421


In [34]:
len(colna_speckle_df.loc[colna_speckle_df["% of Null"] ==100])

26

In [36]:
colna_speckle_df.to_csv(dataprep_path+"NA_Cols_speckle.csv",index=False)

In [37]:
#drop columns with all NA in the speckle data
col100na_speckle_df = colna_speckle_df.loc[colna_speckle_df["% of Null"] ==100]
df = df.loc[:,~df.columns.isin(col100na_speckle_df['Feature'])]
df.shape

(52827, 1634)

In [38]:
#check column na for speckle again
colna_speckle_df =check_colna(df.loc[df['SPECKLE'] == 1])
colna_speckle_df 

,Feature,Total Null,% of Null
1015,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,6484,99.984580
1017,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,6482,99.953739
1043,TPI_VCC::CONT_X_SCREEN_K_FINAL_X_X_X_X_SNR_ADT...,6480,99.922899
1011,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_ADT...,6480,99.922899
1041,TPI_VCC::CONT_X_SCREEN_K_FINAL_X_X_X_X_SNR_ADT...,6480,99.922899
...,...,...,...
1439,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4694,72.382421
1438,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4694,72.382421
1437,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4694,72.382421
1436,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4694,72.382421


#### iii) check 100% nulls in column for non-speckle only and remove them

In [39]:
#check column na for non-speckle
colna_nonspeckle_df =check_colna(df.loc[df['SPECKLE'] == 0])
colna_nonspeckle_df

,Feature,Total Null,% of Null
1019,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,46340,99.995684
1015,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,46332,99.978421
1017,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,46332,99.978421
1013,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,46286,99.879159
1009,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_ADT...,46284,99.874844
...,...,...,...
788,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,32336,69.776876
790,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,32336,69.776876
792,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,32336,69.776876
794,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,32336,69.776876


In [40]:
#drop sort column with string
df["PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110"]
df = df.drop(["PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_CDYN_INDICATOR_CDYN_DATA@132110"],axis=1)

In [41]:
#check column na for whole df after removing columns with 100% NA in whole dataset and speckle data
check_colna(df)

,Feature,Total Null,% of Null
1019,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,52820,99.986749
1015,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,52816,99.979177
1017,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,52814,99.975391
1042,TPI_VCC::CONT_X_SCREEN_K_FINAL_X_X_X_X_SNR_ADT...,52764,99.880743
1011,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_ADT...,52764,99.880743
...,...,...,...
1438,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731
1437,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731
1436,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731
1435,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,37030,70.096731


In [42]:
df.shape

(52827, 1633)

In [ ]:
# colna_df_no100NA.to_csv(path+"/DataPreparation/NA_Cols_without100NA.csv",index=False)

#### b) Check row for 100% nulls and remove them

In [43]:
print(df.shape)
print(df["SPECKLE"].value_counts())

(52827, 1633)
0    46342
1     6485
Name: SPECKLE, dtype: int64


In [ ]:
# pd.options.display.max_seq_items = 2000
# df.columns

In [44]:
supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]', 'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS',
       'INCOMING', 'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS',
       'OUT_BITS', 'DELTA']
rowna_df = check_rowna(df,supporting_fs)
rowna_df

,SPECKLE,VID,Total Null,% of Null
0,1,M00C045U00024,1612,100.00
44219,0,M11T25G601375,1612,100.00
44217,0,M1T59N6700207,1612,100.00
44215,0,M1T59N6700214,1612,100.00
44214,0,M1T59N6700197,1612,100.00
...,...,...,...,...
14096,1,M1CC562301194,61,3.78
27343,0,M17E5T8801464,60,3.72
13034,0,M197674100580,60,3.72
21888,1,M1WE241501785,42,2.61


In [45]:
rowna_df.to_csv(dataprep_path+"NA_Rows.csv",index=False)

In [46]:
#get rows with 100% NA
row100na_df = rowna_df.loc[rowna_df["% of Null"] ==100.00]
row100na_df

,SPECKLE,VID,Total Null,% of Null
0,1,M00C045U00024,1612,100.0
44219,0,M11T25G601375,1612,100.0
44217,0,M1T59N6700207,1612,100.0
44215,0,M1T59N6700214,1612,100.0
44214,0,M1T59N6700197,1612,100.0
...,...,...,...,...
15781,0,M1DY252001984,1612,100.0
12295,0,M17U1C0300361,1612,100.0
50925,0,M1S523K901030,1612,100.0
13053,0,M197674100627,1612,100.0


In [47]:
len(row100na_df)

37030

In [48]:
row100na_df["SPECKLE"].value_counts()

0    32336
1     4694
Name: SPECKLE, dtype: int64

In [49]:
#drop rows with 100% NA (~37k rows)
df = df[~df["VID"].isin(row100na_df["VID"])]
df = df.reset_index(drop=True)
df

,SPECKLE,VID,ULT@MIDAS_6261_U1,SORTLOT,SORTLOT7,WAFER,XLOC,YLOC,ULT@MIDAS_6261_U2,IB@6261[CLASSHOT],...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1,M0118LB500624,N0293010_594_1_9,N0293010,N029301,594,1,9,H0235270_643_8_13,1,...,-7.900000e-07,-7.830000e-07,-8.270000e-07,-1.800000e-07,-2.040000e-07,-1.950000e-07,-2.240000e-07,-7.780000e-07,-7.550000e-07,-1.720000e-07
1,1,M0118LB500644,N0293010_594_3_3,N0293010,N029301,594,3,3,H0235270_643_11_8,1,...,-1.200000e-06,-1.200000e-06,-1.220000e-06,-2.890000e-07,-2.990000e-07,-3.080000e-07,-3.020000e-07,-1.200000e-06,-1.200000e-06,-3.310000e-07
2,1,M0118LB500649,N0293010_594_-3_-5,N0293010,N029301,594,-3,-5,H0235270_643_11_13,1,...,-9.390000e-07,-9.470000e-07,-9.390000e-07,-2.350000e-07,-2.280000e-07,-2.400000e-07,-2.290000e-07,-9.430000e-07,-9.530000e-07,-2.490000e-07
3,1,M0118LB500823,N0293010_594_5_-3,N0293010,N029301,594,5,-3,H0235270_643_11_6,14,...,-1.110000e-06,-1.210000e-06,-1.240000e-06,-2.830000e-07,-2.980000e-07,-3.090000e-07,-2.900000e-07,-1.210000e-06,-1.130000e-06,-2.990000e-07
4,1,M0118LB500903,N0293010_585_-6_-3,N0293010,N029301,585,-6,-3,H0235270_643_14_5,1,...,-6.380000e-07,-6.860000e-07,-6.600000e-07,-1.560000e-07,-1.700000e-07,-1.630000e-07,-1.470000e-07,-6.330000e-07,-6.490000e-07,-1.500000e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15792,0,M1JJ817501233,N0422200_050_-3_1,N0422200,N042220,50,-3,1,H0311010_568_4_-11,1,...,-7.840000e-07,-7.520000e-07,-7.590000e-07,-1.610000e-07,-1.760000e-07,-1.860000e-07,-1.970000e-07,-7.280000e-07,-7.510000e-07,-1.860000e-07
15793,0,M1JJ817501241,N0422200_050_0_4,N0422200,N042220,50,0,4,H0311010_568_4_-8,1,...,-8.900000e-07,-8.520000e-07,-8.660000e-07,-2.130000e-07,-2.040000e-07,-2.200000e-07,-2.290000e-07,-8.970000e-07,-9.040000e-07,-2.050000e-07
15794,0,M1JJ817501242,N0422200_050_0_5,N0422200,N042220,50,0,5,H0311010_568_4_-9,1,...,-7.590000e-07,-7.500000e-07,-7.660000e-07,-1.910000e-07,-1.810000e-07,-1.960000e-07,-1.950000e-07,-8.120000e-07,-8.170000e-07,-1.990000e-07
15795,0,M1JJ817501234,N0422200_050_-3_0,N0422200,N042220,50,-3,0,H0311010_568_4_-12,1,...,-8.870000e-07,-8.530000e-07,-8.720000e-07,-2.170000e-07,-1.960000e-07,-2.090000e-07,-2.180000e-07,-8.270000e-07,-8.250000e-07,-2.320000e-07


In [50]:
df["SPECKLE"].value_counts()

0    14006
1     1791
Name: SPECKLE, dtype: int64

In [51]:
df.shape

(15797, 1633)

In [52]:
#check row na
supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]', 'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS',
       'INCOMING', 'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS',
       'OUT_BITS', 'DELTA']
check_rowna(df,supporting_fs)

,SPECKLE,VID,Total Null,% of Null
4371,1,M1888SS100613,1329,82.44
4377,1,M1888SS100703,1329,82.44
4374,1,M1888SS100619,1329,82.44
4380,1,M1888SS100734,1329,82.44
4373,1,M1888SS100616,1329,82.44
...,...,...,...,...
3315,0,M12C2P5800824,61,3.78
4550,0,M197674100580,60,3.72
12778,0,M17E5T8801464,60,3.72
7805,1,M1WE241501785,42,2.61


In [55]:
df.to_csv(dataprep_path+"df_NAHandling_Part1.csv",index=False)

### 3) Split data into train and validation datasets 

In [56]:
from sklearn.model_selection import train_test_split
from collections import Counter
def randomsamp(df,val_size):
    """
    Split whole dataset into train and validation using random sampling
    Returns X_train, X_val, y_train, y_val
    df[DataFrame]: input dataframe    
    val_size[float]:Should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
                    Train size is complement of val size
    """
    X= df.drop(["SPECKLE"],axis=1)
    y= df["SPECKLE"] 
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = val_size,random_state=42,stratify= df["SPECKLE"])
    if type(val_size)==float:
        print("Train-val split completed with",(1-val_size)*100,"-",val_size*100,"split in train-val")
    print("Shape of X_train is:", X_train.shape)
    print("Shape of X_val is:",X_val.shape)
    print("Shape of y_train is:",y_train.shape)
    print("Shape of y_val is:",y_val.shape)
    print("Distribution of y_train:",Counter(y_train))
    print("Distribution of y_val:",Counter(y_val))
    
    X_train = X_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_val = y_val.reset_index(drop=True)        
           
    return X_train, X_val, y_train, y_val

def targetrandomsamp(df,speckle_test_size,nonspeckle_test_size):
    """
    Dataset is split into speckle/non-speckle first. The speckle/non-speckle datasets are then split 
    into train and validation using random sampling, followed by merging of the speckle/non-speckle  
    to return X_train, X_val, y_train, y_val
    
    df[DataFrame]: input dataframe  
    
    speckle_test_size[float/int]: Val size for speckle 
    -If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
    -If int, represents the absolute number of val samples.
    -Train size is complement of val size
    
    nonspeckle_test_size[float/int]: Val size for non-speckle 
    -If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
    -If int, represents the absolute number of val samples.
    -Train size is complement of val size
    
    """
    #split data into speckle/nonspeckle
    df_s = df.loc[df["SPECKLE"] == 1] #speckle
    df_ns = df.loc[df["SPECKLE"] == 0] #nonspeckle
    
    #split speckle/nonspeckle data into train/validation
    print("For speckle data:")
    X_train_s, X_val_s, y_train_s, y_val_s = randomsamp(df_s,val_size=speckle_test_size)
    print("\nFor non-speckle data:")
    X_train_ns, X_val_ns, y_train_ns, y_val_ns = randomsamp(df_ns,val_size=nonspeckle_test_size)
    
    #concat the speckle/non-speckle train and validation
    X_train = pd.concat([X_train_s,X_train_ns], ignore_index=True)
    X_val = pd.concat([X_val_s,X_val_ns], ignore_index=True)
    y_train = pd.concat([y_train_s,y_train_ns], ignore_index=True)
    y_val = pd.concat([y_val_s,y_val_ns], ignore_index=True)
    
    X_train = X_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_val = y_val.reset_index(drop=True)  
    
    print("\nFinal dataset:")
    print("Distribution of y_train:",Counter(y_train))
    print("Distribution of y_val:",Counter(y_val))
    return X_train, X_val, y_train, y_val

In [57]:
df["SPECKLE"].value_counts()

0    14006
1     1791
Name: SPECKLE, dtype: int64

In [58]:
#train-test split on whole data
X_train, X_val, y_train,y_val = randomsamp(df,val_size=0.3)

Train-val split completed with 70.0 - 30.0 split in train-val
Shape of X_train is: (11057, 1632)
Shape of X_val is: (4740, 1632)
Shape of y_train is: (11057,)
Shape of y_val is: (4740,)
Distribution of y_train: Counter({0: 9803, 1: 1254})
Distribution of y_val: Counter({0: 4203, 1: 537})


In [62]:
pd.concat([X_train,y_train],axis=1).to_csv(dataprep_path+"TrainData.csv",index=False)

In [63]:
pd.concat([X_val,y_val],axis=1).to_csv(dataprep_path+"ValData.csv",index=False)

In [ ]:
#train-test split on speckle/non-speckle then merge
X_train, X_val, y_train, y_val = targetrandomsamp(df,speckle_test_size=500,nonspeckle_test_size=2000)

### 4) NA handling (Part 2 - Remove columns with %NA above a threshold in train data and NA imputation)

#### a) Remove columns with %NA above a threshold in train data

In [64]:
colna_df_no100NA = check_colna(X_train)
colna_df_no100NA

,Feature,Total Null,% of Null
1019,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,11053,99.963824
1015,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,11050,99.936692
1017,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_VCC...,11048,99.918604
1009,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_ADT...,11014,99.611106
1011,TPI_VCC::CONT_X_SCREEN_E_FINAL_X_X_X_X_SNR_ADT...,11014,99.611106
...,...,...,...
930,TPI_VCC::CONT_PARALLEL_VLC_E_FINAL_X_X_X_X_SNR...,9,0.081396
931,TPI_VCC::CONT_PARALLEL_VLC_E_FINAL_X_X_X_X_SNR...,9,0.081396
932,TPI_VCC::CONT_PARALLEL_VLC_E_FINAL_X_X_X_X_SNR...,9,0.081396
933,TPI_VCC::CONT_PARALLEL_VLC_E_FINAL_X_X_X_X_SNR...,9,0.081396


In [65]:
print("No. of columns with NA>= 70%:", len(colna_df_no100NA[colna_df_no100NA["% of Null"] >= 70]))
print("No. of columns with NA>= 80%:", len(colna_df_no100NA[colna_df_no100NA["% of Null"] >= 80]))
print("No. of columns with NA>= 90%:", len(colna_df_no100NA[colna_df_no100NA["% of Null"] >= 90]))

No. of columns with NA>= 70%: 84
No. of columns with NA>= 80%: 84
No. of columns with NA>= 90%: 84


In [66]:
X_train = drop_NAcol(X_train,0.7) #drop columns with >=70% NA 

(11057, 1548)


In [67]:
check_colna(X_train)

,Feature,Total Null,% of Null
741,HVQK_VMIN_PRE_SCAN_UNCORE_VNN@132110,3241,29.311748
1165,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,1986,17.961472
0,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,373,3.373429
399,IDV_0134_PSVT3GVTO12_FULLDIE_TALL_0950_MED@132110,373,3.373429
392,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,373,3.373429
...,...,...,...
896,TPI_VCC::CONT_PARALLEL_VLC_E_FINAL_X_X_X_X_SNR...,9,0.081396
897,TPI_VCC::CONT_PARALLEL_VLC_E_FINAL_X_X_X_X_SNR...,9,0.081396
898,TPI_VCC::CONT_PARALLEL_VLC_E_FINAL_X_X_X_X_SNR...,9,0.081396
899,TPI_VCC::CONT_PARALLEL_VLC_E_FINAL_X_X_X_X_SNR...,9,0.081396


In [68]:
print(X_train.shape)
print(X_val.shape)

(11057, 1548)
(4740, 1632)


In [69]:
#make sure that X_val has same columns as X_train
X_val = X_val.loc[:,X_val.columns.isin(X_train.columns)]

In [70]:
print(X_train.shape)
print(X_val.shape)

(11057, 1548)
(4740, 1548)


### b) NA imputation

In [71]:
def NA_impute(df,imptype):
    """
    Impute NA
    df[DataFrame]:df
    imptype[string]: "mean" to impute data with mean, "median" to impute data with median
    """
    if imptype == "mean":        
        df = df.fillna(df.mean())
    if imptype == "median":
        df = df.fillna(df.median())
    return df

In [72]:
#impute X_train and X_val with median seperately
X_train = NA_impute(X_train,imptype="median")
X_val = NA_impute(X_val,imptype="median")

In [73]:
check_rowna(X_train,supporting_fs) #all NA imputed with median

,SPECKLE,VID,Total Null,% of Null


In [74]:
pd.concat([X_train,y_train],axis=1).to_csv(dataprep_path+"TrainData_NAhandling_Part2.csv",index=False)
pd.concat([X_val,y_val],axis=1).to_csv(dataprep_path+"ValData_NAhandling_Part2.csv",index=False)

### 4) Remove supporting features

In [75]:
supporting_fs = ['VID','ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]', 'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS',
       'INCOMING', 'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS',
       'OUT_BITS', 'DELTA']
X_train_sf = X_train[supporting_fs] #keep supporting fs of X_train
X_val_sf = X_val[supporting_fs] #keep supporting fs of X_val
X_train = X_train.drop(supporting_fs,axis=1) #remove supporting fs from X_train
X_val = X_val.drop(supporting_fs,axis=1) #remove supporting fs from X_val

In [ ]:
print(X_train.shape)
print(X_val.shape)

In [76]:
X_train_sf.to_csv(dataprep_path+"SupFeatures_Train.csv",index=False)
X_val_sf.to_csv(dataprep_path+"SupFeatures_Val.csv",index=False)
pd.concat([X_train,y_train],axis=1).to_csv(dataprep_path+"TrainData_SupFeaturesRem.csv",index=False)
pd.concat([X_val,y_val],axis=1).to_csv(dataprep_path+"ValData_SupFeaturesRem.csv",index=False)

### 3) Handling of negative values (Imputation/conversion)

### a) Converting negative to positive values

In [77]:
def convert_neg_to_pos(df,cols_to_keep):
    """
    df: dataframe
    cols_to_keep[tuple]: tokens for columns which are not converted to negative
    """
    #df with tokens which are not converted to negative
    df_keep = df[df.columns[df.columns.str.startswith(cols_to_keep)]]
    print("df_keep shape",df_keep.shape)
    
    # Create df_to_convert - drop columns from cols_to_keep
    df_to_convert = df.drop([col for col in df if col.startswith(cols_to_keep)], axis=1)    
    print("df_to_convert shape",df_to_convert.shape)
    
    #Convert negative columns in df_to_convert to positive
    df_positive = df_to_convert.abs()
    print("df_positive shape",df_to_convert.shape)
    
    #check whether there's any negative value left in df_positive
    print("Is there negative value left:",(df_positive < 0).any().any())
    
    #Concatenating df_keep and df_positive along columns
    df = pd.concat([df_keep, df_positive], axis=1)
    print("Shape after converting neg to pos:", df.shape)
    
    return df

In [78]:
X_train = convert_neg_to_pos(X_train,cols_to_keep=("IDV", "HVQK"))

df_keep shape (11057, 742)
df_to_convert shape (11057, 786)
df_positive shape (11057, 786)
Is there negative value left: False
Shape after converting neg to pos: (11057, 1528)


In [79]:
X_val = convert_neg_to_pos(X_val,cols_to_keep=("IDV", "HVQK"))

df_keep shape (4740, 742)
df_to_convert shape (4740, 786)
df_positive shape (4740, 786)
Is there negative value left: False
Shape after converting neg to pos: (4740, 1528)


In [ ]:
# ### Create df_keep -> Dataset with supporting features, IDV and HVQK token family columns only  
# #df with supporting fs
# # supporting_fs_df = df[["ULT@MIDAS_6261_U1", "SORTLOT", "SORTLOT7", "WAFER",
# #         "XLOC", "YLOC", "ULT@MIDAS_6261_U2", "IB@6261[CLASSHOT]",
# #         "FB@6261[CLASSHOT]","TEST RESULTS", "TEST RESULTS BITS", "TR_BITS", "INCOMING",
# #         "INCOMING BITS", "INC_BITS", "OUTGOING", "OUTGOING BITS", "OUT_BITS","DELTA","VID"]]
# #df with IDV and HVQK tokens
# cols_to_keep=["IDV", "HVQK"]
# # IDV_HVQK_df = df[df.columns[df.columns.str.startswith(tuple(cols_to_keep))]]
# df_keep = df[df.columns[df.columns.str.startswith(tuple(cols_to_keep))]]
# #Concatenating supporting_fs_df and IDV_HVQK_df along columns
# # df_keep = pd.concat([supporting_fs_df, IDV_HVQK_df], axis=1)
# df_keep.shape

In [ ]:
# # Create df_to_convert - drop columns with supporting features, IDV and HVQK token family columns
# df_to_convert = df.drop([col for col in df if col.startswith('IDV')], axis=1)
# df_to_convert = df_to_convert.drop([col for col in df if col.startswith('HVQK')], axis=1)
# # df_to_convert = df_to_convert.drop(["ULT@MIDAS_6261_U1", "SORTLOT", "SORTLOT7", "WAFER",
# #         "XLOC", "YLOC", "ULT@MIDAS_6261_U2", "IB@6261[CLASSHOT]",
# #         "FB@6261[CLASSHOT]","TEST RESULTS", "TEST RESULTS BITS", "TR_BITS", "INCOMING",
# #         "INCOMING BITS", "INC_BITS", "OUTGOING", "OUTGOING BITS", "OUT_BITS","DELTA","VID"], axis=1)
# df_to_convert.shape

In [ ]:
#Convert negative columns in df_to_convert to positive
# df_positive = df_to_convert.abs()
# df_positive.shape

In [ ]:
# #check whether there's any negative value left in df_positive
# (df_positive < 0).any().any()

In [ ]:
# #Concatenating df_keep and df_positive along columns
# df = pd.concat([df_keep, df_positive], axis=1)
# df.shape

### b) Imputing negative values

In [80]:
#Checking column for negative values
def check_col_negative(df,value):
    """
    Check each column for negative values. Returns Feature, total negative values and % of negative values for each column
    df[DataFrame]: dataframe
    
    """
    col_negative_df = pd.DataFrame(columns =["Feature", "Total Negative Values", "% of Negative Values"])
    for col in df.columns: 
        #checking if there is any specific negative value in the column

        if df[col].isin([value]).sum()>0: 
            
            # if specific negative value present, total number of specific negative value in the column stores here
            total_negative = df[col].isin([value]).sum() 
            new_row = {'Feature':col, 'Total Negative Values':total_negative, '% of Negative Values':round(total_negative*100/len(df),2)}
            #append row to the dataframe
            col_negative_df = col_negative_df.append(new_row, ignore_index=True)
            
    col_negative_df = col_negative_df.sort_values("% of Negative Values", ascending=False)    
    return col_negative_df  

In [81]:
# X_train
print("Checking column for -5555 values")
display(check_col_negative(X_train,value= -5555))
print("Checking column for -555 values")
display(check_col_negative(X_train,value= -555))
print("Checking column for -999 values")
display(check_col_negative(X_train,value= -999))
print("Checking column for -9999 values")
display(check_col_negative(X_train,value = -9999))

Checking column for -5555 values


,Feature,Total Negative Values,% of Negative Values
5,HVQK_VMIN_POST_ARRAY_NAC_MBIST_LSACPMCPR@132110,1571,14.21
43,HVQK_VMIN_POST_SCAN_NAC_CPMC@132110,1571,14.21
67,HVQK_VMIN_PRE_SCAN_NAC_CPMC@132110,1177,10.64
6,HVQK_VMIN_POST_ARRAY_NAC_MBIST_LSACPMPKE@132110,727,6.58
42,HVQK_VMIN_POST_SCAN_NAC_CPMB@132110,727,6.58
...,...,...,...
70,HVQK_VMIN_PRE_SCAN_UNCORE_LLCSFP11@132110,261,2.36
71,HVQK_VMIN_PRE_SCAN_UNCORE_LLCSFP12@132110,232,2.10
73,HVQK_VMIN_PRE_SCAN_UNCORE_LLCSFP23@132110,230,2.08
72,HVQK_VMIN_PRE_SCAN_UNCORE_LLCSFP21@132110,226,2.04


Checking column for -555 values


,Feature,Total Negative Values,% of Negative Values


Checking column for -999 values


,Feature,Total Negative Values,% of Negative Values
0,IDV_0101_SVT3GNES12_FULLDIE_CORE_TALL_0650_MED...,601,5.44
37,IDV_0147_NOMNLK12_FULLDIE_CORE_TALL_0650_MED@1...,601,5.44
27,IDV_0137_PNOM3GVTO12_FULLDIE_CORE_TALL_0650_ME...,601,5.44
28,IDV_0138_PNOM3GVTO12_FULLDIE_CORE_TALL_0650_ME...,601,5.44
29,IDV_0139_NSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,601,5.44
30,IDV_0140_NSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,601,5.44
31,IDV_0141_NSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,601,5.44
32,IDV_0142_NSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,601,5.44
33,IDV_0143_NNOM3GVTO12_FULLDIE_CORE_TALL_0650_ME...,601,5.44
34,IDV_0144_NNOM3GVTO12_FULLDIE_CORE_TALL_0650_ME...,601,5.44


Checking column for -9999 values


,Feature,Total Negative Values,% of Negative Values
0,HVQK_VMIN_PRE_ARRAY_NAC_MBIST_LSACPMCPR@132110,700,6.33
3,HVQK_VMIN_PRE_SCAN_NAC_CPMC@132110,237,2.14
2,HVQK_VMIN_PRE_SCAN_NAC_CPMB@132110,101,0.91
1,HVQK_VMIN_PRE_ARRAY_NAC_MBIST_LSACPMPKE@132110,64,0.58


In [82]:
# X_val
print("Checking column for -5555 values")
display(check_col_negative(X_val,value= -5555))
print("Checking column for -555 values")
display(check_col_negative(X_val,value= -555))
print("Checking column for -999 values")
display(check_col_negative(X_val,value= -999))
print("Checking column for -9999 values")
display(check_col_negative(X_val,value = -9999))

Checking column for -5555 values


,Feature,Total Negative Values,% of Negative Values
43,HVQK_VMIN_POST_SCAN_NAC_CPMC@132110,687,14.49
5,HVQK_VMIN_POST_ARRAY_NAC_MBIST_LSACPMCPR@132110,687,14.49
67,HVQK_VMIN_PRE_SCAN_NAC_CPMC@132110,538,11.35
42,HVQK_VMIN_POST_SCAN_NAC_CPMB@132110,337,7.11
6,HVQK_VMIN_POST_ARRAY_NAC_MBIST_LSACPMPKE@132110,337,7.11
...,...,...,...
69,HVQK_VMIN_PRE_SCAN_UNCORE_LLCSFP02@132110,100,2.11
72,HVQK_VMIN_PRE_SCAN_UNCORE_LLCSFP21@132110,100,2.11
70,HVQK_VMIN_PRE_SCAN_UNCORE_LLCSFP11@132110,95,2.00
71,HVQK_VMIN_PRE_SCAN_UNCORE_LLCSFP12@132110,86,1.81


Checking column for -555 values


,Feature,Total Negative Values,% of Negative Values


Checking column for -999 values


,Feature,Total Negative Values,% of Negative Values
0,IDV_0101_SVT3GNES12_FULLDIE_CORE_TALL_0650_MED...,269,5.68
37,IDV_0147_NOMNLK12_FULLDIE_CORE_TALL_0650_MED@1...,269,5.68
27,IDV_0137_PNOM3GVTO12_FULLDIE_CORE_TALL_0650_ME...,269,5.68
28,IDV_0138_PNOM3GVTO12_FULLDIE_CORE_TALL_0650_ME...,269,5.68
29,IDV_0139_NSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,269,5.68
30,IDV_0140_NSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,269,5.68
31,IDV_0141_NSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,269,5.68
32,IDV_0142_NSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,269,5.68
33,IDV_0143_NNOM3GVTO12_FULLDIE_CORE_TALL_0650_ME...,269,5.68
34,IDV_0144_NNOM3GVTO12_FULLDIE_CORE_TALL_0650_ME...,269,5.68


Checking column for -9999 values


,Feature,Total Negative Values,% of Negative Values
0,HVQK_VMIN_PRE_ARRAY_NAC_MBIST_LSACPMCPR@132110,337,7.11
3,HVQK_VMIN_PRE_SCAN_NAC_CPMC@132110,81,1.71
2,HVQK_VMIN_PRE_SCAN_NAC_CPMB@132110,44,0.93
1,HVQK_VMIN_PRE_ARRAY_NAC_MBIST_LSACPMPKE@132110,30,0.63


In [83]:
#Negative value imputation
def Negative_value_impute(df,value,imptype):
    """
    Impute Negative value (can choose negative value to impute)
    df[DataFrame]:df
    imptype[string]: "mean" to impute data with mean, "median" to impute data with median
    """
    if imptype == "mean":        
        df = df.replace(value,df.mean())
    if imptype == "median":
        df = df.replace(value,df.median())
    return df

In [84]:
#X_train imputation
#Negative -5555 imputation with median
X_train = Negative_value_impute(X_train,value=-5555,imptype="median")
#Negative 999 imputation with median
X_train = Negative_value_impute(X_train,value=-999,imptype="median")
#Negative 9999 imputation with median
X_train = Negative_value_impute(X_train,value=-9999,imptype="median")
#Negative -555 imputation with median
X_train = Negative_value_impute(X_train,value=-555,imptype="median")
X_train.shape

(11057, 1528)

In [85]:
#X_val imputation
#Negative -5555 imputation with median
X_val = Negative_value_impute(X_val,value=-5555,imptype="median")
#Negative 999 imputation with median
X_val = Negative_value_impute(X_val,value=-999,imptype="median")
#Negative 9999 imputation with median
X_val = Negative_value_impute(X_val,value=-9999,imptype="median")
#Negative -555 imputation with median
X_val = Negative_value_impute(X_val,value=-555,imptype="median")
X_val.shape

(4740, 1528)

In [87]:
#To check if all negative values have been converted
#Filter only numeric data
df_numeric = X_train._get_numeric_data() #change to X_train or X_val
#Filter columns with negative values
df_negative = df_numeric.loc[:,(df_numeric < 0).any()]
df_negative

""
0
1
2
3
4
...
11052
11053
11054
11055


In [88]:
pd.concat([X_train,y_train],axis=1).to_csv(dataprep_path+"TrainData_NegHandling.csv",index=False)
pd.concat([X_val,y_val],axis=1).to_csv(dataprep_path+"ValData_NegHandling.csv",index=False)

### 4) Check and remove unary(single value) columns

In [89]:
def unary(df):
    """
    Checks for unary columns. If there are unary columns, the unary columns will be printed and removed from the df
    df[DataFrame]: input dataframe
    """
    unarycolumns = [col for col in df.columns if len(df[col].unique())==1]
    if unarycolumns:
        print("The unary column are:",unarycolumns)
        df = df.drop(unarycolumns,axis=1)
        print("Unary columns dropped!")
        return df 
    else:
        print("There are no unary columns!")      


In [90]:
unary(X_train)

There are no unary columns!


In [91]:
unary(X_val)

There are no unary columns!


### 7) Scaling the data (Standardization/Normalization)

In [92]:
#To scale X train and X validation (test) data using various scalers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def scale_data(X_train,X_val,scaler_type):
    """
    Scaling X train and validation with standardization or normalization
    params:
    X_train[DataFrame]: input X train
    X_val[DataFrame]: input X validation (test)
    scaler_type[None/string]: input scaling method
    - "Standardization" for Standard Scaler
    - "Normalization" for Min Max Scaler    

    """           
    if scaler_type == "Standardization":
        scaler = StandardScaler()
    if scaler_type == "Normalization":
        scaler = MinMaxScaler()

    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled,columns= X_train.columns)
    X_val_scaled = scaler.transform(X_val)
    X_val_scaled = pd.DataFrame(X_val_scaled,columns= X_val.columns)
       
    return X_train_scaled, X_val_scaled

In [93]:
#X train-test scaled using Min max Scaler
X_train_scaled, X_val_scaled = scale_data(X_train,X_val,scaler_type = "Normalization")

In [94]:
pd.concat([X_train_scaled,y_train],axis=1).to_csv(dataprep_path+"TrainData_Normalized.csv",index=False)
pd.concat([X_val_scaled,y_val],axis=1).to_csv(dataprep_path+"ValData_Normalized.csv",index=False)

In [166]:
# X_train_scaled = pd.read_csv(dataprep_path+"TrainData_Normalized.csv")
# X_train_scaled = X_train_scaled.drop(["SPECKLE"],axis=1)
# X_val_scaled = pd.read_csv(dataprep_path+"ValData_Normalized.csv")
# X_val_scaled = X_val_scaled.drop(["SPECKLE"],axis=1)

In [167]:
print(X_train_scaled.shape)
print(y_train.shape)
print(X_val_scaled.shape)
print(y_val.shape)

(11057, 1528)
(11057,)
(4740, 1528)
(4740,)


### 8) Handling highly correlated features (features vs features/features vs target)

#### a) features vs features

In [95]:
%%time
import numpy as np

#To create correlation matrix for X_train (features)
correlation_matrix = X_train_scaled.corr().abs()

#To select the upper trigular matrix from the correlation matrix of features
upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape),k=1).astype(np.bool))
display(upper_tri)

#features with more than or equal to 95% correlation
highly_corr_features_95 = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]


,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,NaN,0.868735,0.987811,0.859960,0.993205,0.864417,0.988577,0.860200,0.966425,0.880205,...,0.309225,0.309365,0.303720,0.321451,0.312548,0.299867,0.320903,0.317835,0.319566,0.298835
IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,NaN,NaN,0.845181,0.978720,0.862040,0.990497,0.846372,0.979926,0.784824,0.978133,...,0.123406,0.121733,0.116536,0.149820,0.135316,0.123537,0.140998,0.131479,0.135537,0.118905
IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,NaN,NaN,NaN,0.862398,0.988959,0.848488,0.997316,0.859266,0.968519,0.866728,...,0.330523,0.330121,0.324665,0.343593,0.333427,0.322019,0.342569,0.340962,0.342133,0.321841
IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,NaN,NaN,NaN,NaN,0.860471,0.980166,0.860335,0.995952,0.783838,0.966615,...,0.120622,0.118516,0.113136,0.149009,0.132428,0.122022,0.140105,0.130709,0.134039,0.117868
IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,NaN,NaN,NaN,NaN,NaN,0.870077,0.988355,0.858725,0.967864,0.880151,...,0.314842,0.314845,0.309371,0.328022,0.318472,0.306210,0.327540,0.323785,0.325280,0.304821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.944578,0.959700,0.958325,0.937627
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.958337,0.956548,0.934293
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.990199,0.955882
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.954153


Wall time: 1min 31s


In [96]:
len(highly_corr_features_95)

928

In [114]:
#get the highly corr features to be removed
highly_corr_features_df = pd.DataFrame (highly_corr_features_95, columns = ['Feature'])
highly_corr_features_df
highly_corr_features_df.to_csv(dataprep_path+"highly_corr_features_0.95.csv",index=False)

In [105]:
#Display correlation matrix pairs (sorted in descending order)
# correlation_matrix_stack = X_train_scaled.corr().abs()

#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)
# upper_tri_stack = (correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
#                   .stack()
#                   .sort_values(ascending=False))
# #first element of upper_tri_stack series is the pair with the biggest correlation
# upper_tri_stack

PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_PT_CDYN_VCCCORE_M2_0P95_1600@132110                 PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_PT_CDYN_VCCCORE_M4_0P95_1600@132110           1.000000e+00
PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_PT_CDYN_VCCCORE_M1_0P95_1600@132110                 PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_PT_CDYN_VCCCORE_M5_0P95_1600@132110           1.000000e+00
PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_PT_CDYN_VCCCORE_M2_0P95_1600@132110                 PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_PT_CDYN_VCCCORE_M3_0P95_1600@132110           1.000000e+00
PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_PT_CDYN_VCCCORE_M0_0P95_1600@132110                 PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_PT_CDYN_VCCCORE_M1_0P95_1600@132110           1.000000e+00
                                                                                                      PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC_PP_PT_C

In [151]:
# the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)
upper_tri_stack = (correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
                  .stack()
                  .sort_values(ascending=False))

#Create empty df
column_names = ["Feature 1", "Feature 2", "Correlation"]
results = pd.DataFrame(columns = column_names)                   

for index, value in upper_tri_stack.items(): 
    if value >= 0.95:
        new_row = {'Feature 1':index[0], 'Feature 2':index[1],'Correlation':value}
        #append row to the dataframe
        results = results.append(new_row, ignore_index=True)

results = results.sort_values(by ='Correlation', axis = 0,ascending=False)         
results

,Feature 1,Feature 2,Correlation
0,PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC...,PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC...,1.000000
8,PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC...,PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC...,1.000000
14,PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC...,PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC...,1.000000
13,PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC...,PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC...,1.000000
12,PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC...,PTH_POWER::POWER_X_SCREEN_E_BEGIN_X_X_X_X_CALC...,1.000000
...,...,...,...
10209,IDV_0059_NOM3GAGN12_FULLDIE_SOC_TALL_0650_MED@...,IDV_0104_NOM3GNES12_FULLDIE_CORE_TALL_0650_MED...,0.950016
10210,IDV_0103_SVT3GSEM12_FULLDIE_CORE_TALL_0950_MED...,IDV_0150_SVT2GNES8_FULLDIE_CORE_TALL_0950_MED@...,0.950014
10211,IDV_0045_REFOMVTO12_FULLDIE_SOC_TALL_0650_MED@...,IDV_0144_NNOM3GVTO12_FULLDIE_SOC_TALL_0950_MED...,0.950013
10212,IDV_0108_SVT1GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0112_SVT1GISO12_FULLDIE_SOC_0950_MED@132110,0.950011


In [153]:
results.to_csv(dataprep_path+"highly_corr_features_0.95_pairs.csv",index=False)

In [168]:
#Dropping the columns with high correlation
X_train_scaled = X_train_scaled.drop(highly_corr_features_95, axis=1)
X_train_scaled.shape

(11057, 600)

In [169]:
pd.concat([X_train_scaled,y_train],axis=1).to_csv(dataprep_path+"TrainData_Corr0.95Rem.csv",index=False)

#### b) features vs target

In [170]:
#To create correlation matrix for X_train (features) and y_train (target)
correlation_matrix_target = pd.DataFrame(X_train_scaled.corrwith(y_train, axis = 0).abs(),columns = ["Correlation"])
correlation_matrix_target.reset_index(inplace=True)
correlation_matrix_target = correlation_matrix_target.rename(columns = {'index':'Feature'}).sort_values(by="Correlation",ascending=False)
correlation_matrix_target

,Feature,Correlation
90,HVQK_VMIN_POST_ARRAY_CORE_PBIST_L2SSA@132110,0.375291
91,HVQK_VMIN_POST_ARRAY_CORE_PBIST_L2SSAC6S@132110,0.301972
28,IDV_0048_SVTNLK12_FULLDIE_SOC_0950_MED@132110,0.292620
272,TPI_VCC::CONT_PARALLEL_LC_E_FINAL_X_X_X_X_SNR_...,0.271651
322,TPI_VCC::CONT_VCCCOREM45_HC_K_START_X_X_X_X_PO...,0.265944
...,...,...
591,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,0.000600
353,TPI_VCC_X::CONT_PARALLEL_HC_E_START_X_X_X_X_MI...,0.000489
526,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,0.000402
86,IDV_0235_NOMMUXFEM12_FULLDIE_0650_MED@132110,0.000355


In [171]:
correlation_matrix_target.to_csv(dataprep_path+"FeaturesVSTarget_Corr.csv",index=False)

### B) Sampling to address class imbalance

In [ ]:
Counter(y_train)

In [ ]:
from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter

#SMOTE
def SMOTE_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose SMOTE or SMOTE + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    over_amt [int] : amount of synthetic data to be generated with SMOTE (speckle)
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #SMOTE only
        sm = SMOTE(sampling_strategy= {1:over_amt},random_state=42)
        X_s, y_s = sm.fit_resample(X_train_scaled, y_train)
        
    else: #SMOTE + random undersampling
        sm = SMOTE(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', sm), ('u', under)])
        X_s, y_s = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_s))     
    print("Ratio of class distribution after sampling :",round(Counter(y_s)[0]/Counter(y_s)[1],2)) 
    
    return X_s, y_s

#BorderlineSMOTE
def BorderlineSMOTE_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose BorderlineSMOTE or BorderlineSMOTE + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    supporting_fs[list] : list of supporting features to be removed from X_train_scaled
    over_amt [int] : amount of synthetic data to be generated with BorderlineSMOTE
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #BorderlineSMOTE only
        bsm = BorderlineSMOTE(sampling_strategy= {1:over_amt},random_state=42)
        X_bs, y_bs = bsm.fit_resample(X_train_scaled, y_train)
        
    else: #BorderlineSMOTE + random undersampling
        bsm = BorderlineSMOTE(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', bsm), ('u', under)])
        X_bs, y_bs = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_bs))     
    print("Ratio of class distribution after sampling :",round(Counter(y_bs)[0]/Counter(y_bs)[1],2)) 
    
    return X_bs, y_bs

#ADASYN
def ADASYN_sampling(X_train_scaled,y_train,over_amt,under_amt=None):
    """
    Can choose ADASYN or ADASYN + random undersampling
    X_train_scaled[DataFrame]: X_train after scaling
    y_train[DataFrame] : target in train data
    supporting_fs[list] : list of supporting features to be removed from X_train_scaled
    over_amt [int] : amount of synthetic data to be generated with ADASYN
    under_amt[int/None]: int - amount of nonspeckle data after undersampling
                         None - No undersampling is done
    
    """
    print("Class distribution before sampling:", Counter(y_train))
    print("Ratio of class distribution before sampling :",round(Counter(y_train)[0]/Counter(y_train)[1],2))
    
    if under_amt == None: #ADASYN only
        ad = ADASYN(sampling_strategy= {1:over_amt},random_state=42)
        X_a, y_a = ad.fit_resample(X_train_scaled, y_train)
        
    else: #ADASYN + random undersampling
        ad = ADASYN(sampling_strategy={1:over_amt},random_state=42)
        under = RandomUnderSampler(sampling_strategy={0:under_amt},random_state=42)
        pipeline = Pipeline(steps=[('o', ad), ('u', under)])
        X_a, y_a = pipeline.fit_resample(X_train_scaled, y_train)    
         
    print("Class distribution after sampling:", Counter(y_a))     
    print("Ratio of class distribution after sampling :",round(Counter(y_a)[0]/Counter(y_a)[1],2)) 
    
    return X_a, y_a

In [ ]:
#SMOTE
X_s, y_s = SMOTE_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=2000)      

In [ ]:
#BorderlineSMOTE
X_bs, y_bs = BorderlineSMOTE_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=2000)      

In [ ]:
#ADASYN
X_a, y_a = ADASYN_sampling(X_train_scaled,y_train,over_amt=2000,under_amt=None)      

In [ ]:
#create new column for target
# import numpy as np
# df["SPECKLE"] = np.where(df["DELTA"]==0,0,1)
# cols = df.columns.tolist()
# cols = [cols[-1]]+cols[:-1] #move speckle col to the front
# df = df.reindex(columns=cols)

In [ ]:
# df.to_csv(path+"SNR_R2_ww35.2_Speckle.csv",index=False)

In [ ]:
# supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
#        'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
#        'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
#        'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
#        'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
# df1 = df.drop(supporting_fs,axis=1) 
# df1.head()

### 3) Feature Selection

In [4]:
#make folder for featureselection if it doesn't exist
import os
featuresel_path = path + 'FeatureSelection/SNR_R5_ww51.4/' 
if not os.path.exists(featuresel_path):
    os.makedirs(featuresel_path)
    print("FeatureSelection folder created!")

#### a) ANOVA

In [160]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

def anova_fs(X_train_scaled,y_train):
    """
    Anova feature selection. Return the top k features 
    X_train_scaled[df]: X of train data
    y_train[df]: y of train data (target)
    num_top_fs[int/"all"]: int -> specify the number of top fs to be returned,
                                  returns top fs and their scores in descending order
                           "all" -> returns all fs and their scores in descending order
    """
    #Select k features with top Fisher scores
    anova = SelectKBest(f_classif, k = "all")    
    anova.fit(X_train_scaled, y_train)
            
    #get the top feature names
    fs_names = X_train_scaled.columns.values[anova.get_support()] 

    #get the top feature scores
    scores = anova.scores_[anova.get_support()] 
    
    #dataframe with feature name and score in descending order
    names_scores = list(zip(fs_names, scores))
    ns_df = pd.DataFrame(data = names_scores, columns=['Feature', 'Scores'])
    ns_df = ns_df.sort_values(by=['Scores'], ascending=False)    
    ns_df['Scores_Scaled'] = ns_df['Scores']/ns_df['Scores'].sum()
    ns_df['Scores_Scaled_Cumulative %'] = ns_df['Scores_Scaled'].cumsum()*100
    ns_df = ns_df.reset_index(drop=True)
    
    return anova, ns_df

In [172]:
anova,ns_df = anova_fs(X_train_scaled,y_train)
print(anova)
ns_df

SelectKBest(k='all')


,Feature,Scores,Scores_Scaled,Scores_Scaled_Cumulative %
0,HVQK_VMIN_POST_ARRAY_CORE_PBIST_L2SSA@132110,1812.272587,2.419708e-02,2.419708
1,HVQK_VMIN_POST_ARRAY_CORE_PBIST_L2SSAC6S@132110,1109.222065,1.481010e-02,3.900719
2,IDV_0048_SVTNLK12_FULLDIE_SOC_0950_MED@132110,1035.242133,1.382234e-02,5.282952
3,TPI_VCC::CONT_PARALLEL_LC_E_FINAL_X_X_X_X_SNR_...,880.794151,1.176018e-02,6.458970
4,TPI_VCC::CONT_VCCCOREM45_HC_K_START_X_X_X_X_PO...,841.385417,1.123400e-02,7.582370
...,...,...,...,...
595,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,0.003975,5.307695e-08,99.999992
596,TPI_VCC_X::CONT_PARALLEL_HC_E_START_X_X_X_X_MI...,0.002638,3.522783e-08,99.999996
597,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,0.001785,2.383607e-08,99.999998
598,IDV_0235_NOMMUXFEM12_FULLDIE_0650_MED@132110,0.001390,1.856348e-08,100.000000


In [173]:
ns_df.to_csv(featuresel_path+"Anova_Fs.csv",index=False)

In [174]:
#take top n important features only in train and validation
n=200
features = ns_df.head(n)[["Feature"]] #choose the top n important features
X_train_scaled = X_train_scaled[features["Feature"]] #filter X_train_scaled with top n important features
X_val_scaled = X_val_scaled[features["Feature"]] #filter X_val_scaled with top n important features

In [175]:
print(X_train_scaled.shape)
print(X_val_scaled.shape)

(11057, 200)
(4740, 200)


### b) Boruta

In [ ]:
X_train_scaled.shape

In [ ]:
y_train.value_counts()

In [ ]:
%%time

# Build the grid search on random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

param_grid = {
#     'n_estimators': [100, 300], # number of trees, default is 100 -> not used as Boruta will determine
    'max_depth': [3,5,7], #max_depth of the tree is advised on the Boruta Github page to be between 3 to 7
    'max_features': ['auto','log2'], #number of features to consider when looking for the best split, default auto
    'min_samples_leaf': [1,100], # minimum sample leaf (decision tree end node) size, default is 1
    'class_weight': [None,'balanced'], #weights for unbalanced data     
    'random_state': [42]
}

rf_model = RandomForestClassifier() 
grid = GridSearchCV(estimator=rf_model, param_grid=param_grid,scoring='balanced_accuracy',n_jobs = -1,verbose=2,cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 

print("Best parameters:", grid_results.best_params_)

#model with best parameters
model = grid_results.best_estimator_ 
print(model)

fs_importance= pd.DataFrame(list(zip(X_train_scaled.columns, model.feature_importances_)), columns=['Feature', 'Importance']).sort_values(by=['Importance'], ascending=False)  
display(fs_importance)

In [ ]:
results_df = pd.DataFrame(grid_results.cv_results_)
results_df= results_df.drop(["mean_fit_time","std_fit_time","mean_score_time","std_score_time","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score"],axis=1)
results_df

In [ ]:
rf_fs_importance.to_csv(featuresel_path+"RandomForest.csv",index=False)

In [ ]:
# rf_fs_importance["Importance"].sum()

In [ ]:
%%time
# Creating a BorutaPy object with RandomForestClassifier as the estimator and ranking the features
from boruta import BorutaPy
boruta_selector = BorutaPy(best_rf_model, n_estimators='auto', verbose=2, perc = 90,max_iter = 100, random_state=42)                                                      
boruta_selector.fit(np.array(X_train_scaled), np.array(y_train))

In [ ]:
print(boruta_selector)
print("No. of significant features: ", boruta_selector.n_features_) 
selected_boruta_fs = pd.DataFrame({'Feature':list(X_train_scaled.columns),'Ranking':boruta_selector.ranking_}).sort_values(by='Ranking')
selected_boruta_fs = selected_boruta_fs[selected_boruta_fs['Ranking']==1]
selected_boruta_fs = selected_boruta_fs[["Feature"]]
selected_boruta_fs_imp = rf_fs_importance.loc[rf_fs_importance['Feature'].isin(selected_boruta_fs['Feature']),:]
selected_boruta_fs_imp

In [ ]:
selected_boruta_fs_imp.to_csv(featuresel_path+"Boruta.csv",index=False)

In [ ]:
#take important features only in train and validation
X_train_scaled = X_train_scaled[selected_boruta_fs_imp["Feature"]]
X_val_scaled = X_val_scaled[selected_boruta_fs_imp["Feature"]]

### c) Random Forest

In [ ]:
X_train_scaled.shape

In [ ]:
y_train.value_counts()

In [ ]:
# Build the grid search
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

start = time.time()

param_grid = {
    'n_estimators': [100, 300], # number of trees, default is 100
    'max_features': ['auto','log2'], #number of features to consider when looking for the best split, default auto
    'min_samples_leaf': [1,100], # minimum sample leaf (decision tree end node) size, default is 1
    'class_weight': [None,'balanced'], #weights for unbalanced data     
    'random_state': [42]
}

# Grid search on random forest
rf_model = RandomForestClassifier() 
grid = GridSearchCV(estimator=rf_model, param_grid=param_grid,scoring='balanced_accuracy',n_jobs = -1,verbose=2,cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 


# # Print the time spend and number of models ran
# print("GridSearchCV took %.2f seconds for %d candidate parameter settings." % ((time.time() - start), len(grid_results.cv_results_['params'])))

end = time.time()
print ("Time elapsed:", end - start)

In [ ]:
print("Best parameters:", grid_results.best_params_)
#model with best parameters
best_rf_model = grid_results.best_estimator_ 
best_rf_model

In [ ]:
rf_fs_importance= pd.DataFrame(list(zip(X_train_scaled.columns, best_rf_model.feature_importances_)), columns=['Feature', 'Importance']).sort_values(by=['Importance'], ascending=False)  
rf_fs_importance

In [ ]:
results_df = pd.DataFrame(grid_results.cv_results_)
results_df= results_df.drop(["mean_fit_time","std_fit_time","mean_score_time","std_score_time","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score"],axis=1)
results_df

### d) XGBoost

In [ ]:
# Build the grid search
from xgboost import XGBClassifier

start = time.time()

param_grid = {
    'n_estimators': [100, 600],
    'learning_rate': [0.01],
    'min_child_weight': [1, 10],
    'gamma': [0.5, 5],
    'subsample': [0.5, 1.0],
    'colsample_bytree': [0.5, 1.0],
    'max_depth': [3, 5],
    'objective': ['binary:logistic'],
    'nthread': [1],
    'verbosity': [0],
    'random_state': [42]
}

# Grid search on XGBoost
xgb_model = XGBClassifier()
grid = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 

# # Print the time spend and number of models ran
# print("GridSearchCV took %.2f seconds for %d candidate parameter settings." % ((time.time() - start), len(grid_results.cv_results_['params'])))

end = time.time()
print ("Time elapsed:", end - start)

In [ ]:
print("Best parameters:", grid_results.best_params_)
#model with best parameters
best_xgb_model = grid_results.best_estimator_ 
best_xgb_model

In [ ]:
xgb_fs_importance = pd.DataFrame(list(zip(X_train_scaled.columns, best_xgb_model.feature_importances_)), columns=['Feature', 'Importance']).sort_values(by=['Importance'], ascending=False)  
xgb_fs_importance

In [ ]:
results_df = pd.DataFrame(grid_results.cv_results_)
results_df= results_df.drop(["mean_fit_time","std_fit_time","mean_score_time","std_score_time","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score"],axis=1)
results_df

### f) Ensemble feature selection

In [ ]:
best_rf_model

In [ ]:
best_xgb_model

In [ ]:
%%time
anova._estimator_type = "classifier"
from xgboost import XGBClassifier
best_xgb_model = XGBClassifier(random_state = 42)

#ensemble feature selection
from sklearn.ensemble import VotingClassifier
#specify the estimators to use, specify anova first if it is used
estimators = [('ANOVA',anova),('XGB',best_xgb_model),('RF', best_rf_model)] 
voting_clf = VotingClassifier(estimators=estimators, voting='soft', weights=[1,1,1])
voting_clf.fit(X_train_scaled, y_train)

In [ ]:
def compute_feature_importance(voting_clf, weights):
    """ 
    Function to compute feature importance of ensemble feature selection 
    voting_clf = Voting Classifier object
    weights = weights used for each estimator 
    """
    
    feature_importance = dict()
    anova_featureimp = dict()

    try: # anova is not one of the estimators
        for est in voting_clf.estimators_:
            feature_importance[str(est)] = est.feature_importances_ #get the feature imp for each estimator
    except: #if anova is one of the estimators
        #get the feature imp for anova
        anova_featureimp[voting_clf.estimators_[0]] = voting_clf.estimators_[0].scores_/voting_clf.estimators_[0].scores_.sum()
        #get the feature imp for other estimators
        for est in voting_clf.estimators_[1:]: 
            feature_importance[str(est)] = est.feature_importances_ #get the feature imp for each estimator    
        #combine feature imp of anova and other estimators
        feature_importance.update(anova_featureimp)
    
    fe_scores = [0]*len(list(feature_importance.values())[0]) #initialize fe_scores as 0 
    for idx, imp_score in enumerate(feature_importance.values()): #idx:estimator index, imp_score: feature imp of est
        imp_score_with_weight = imp_score*weights[idx] #Multiply the weights of the base estimator to the importance score of each of the features.
        fe_scores = list(np.add(fe_scores, list(imp_score_with_weight)))
        
    return fe_scores

ensemble_fs = pd.DataFrame()
ensemble_fs['Feature'] = X_train_scaled.columns
ensemble_fs['Feature Importance'] = compute_feature_importance(voting_clf,weights = [1, 1, 1])
ensemble_fs = ensemble_fs.sort_values('Feature Importance', ascending=False)
ensemble_fs

In [ ]:
#take top n important features only in train and validation
n=200
features = ns_df.head(n)[["Feature"]] #choose the top n important features
X_train_scaled = X_train_scaled[features["Feature"]] #filter X_train_scaled with top n important features
X_val_scaled = X_val_scaled[features["Feature"]] #filter X_val_scaled with top n important features

In [ ]:
ensemble_fs.to_csv(featuresel_path+"Ensemble_Anova_XGB_RF.csv",index=False)

In [ ]:
# df2 = pd.read_csv(path+'DataPreparation/negative_parameters.csv')

In [ ]:
# df2.head()

In [ ]:
# df_IDV = df2.loc[:, df2.columns.str.startswith("IDV")]
# df_IDV.head()

In [ ]:
# df_IDV = df2.loc[:, df2.columns.str.startswith("IDV")]
# df_HVQK = df2.loc[:, df2.columns.str.startswith("HVQK")]
# df_IDV.min()
# df_HVQK.min()

In [ ]:
# df_HVQK = df2.loc[:, df2.columns.str.startswith("HVQK")]
# df_HVQK.head()

In [ ]:
# df_IDV.min()

In [ ]:
# df_HVQK.min()

In [ ]:
# df2.loc[:,((df2 == -5555).any()) & ((df2 == -9999).any())]

In [ ]:
# import pandas as pd
# path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/ML based speckle POC/"
# df2 = pd.read_csv(path+"SNR_R5_ww51.4.csv")
# df3 = df2[df2["VID"].isin(row100na_df["VID"])]
# df3 = df3.reset_index(drop=True)
# df3.shape
# df3.to_csv(path+"/DataPreparation/NA_rows_withsf.csv",index=False)

### Model Building

In [176]:
print(X_train_scaled.shape)
print(X_val_scaled.shape)

(11057, 200)
(4740, 200)


In [178]:
print(y_train.value_counts())
print(y_val.value_counts())

0    9803
1    1254
Name: SPECKLE, dtype: int64
0    4203
1     537
Name: SPECKLE, dtype: int64


### a) SVM


In [179]:
%%time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C':[0.1, 1, 10, 100, 1000], #Regularization parameter [0.1, 1, 10, 100, 1000]
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'], #Specifies the kernel type to be used in the algorithm
    'gamma': ['scale', 'auto'],
    'class_weight': [None,'balanced'], #weights for unbalanced data     
    'random_state': [42]
}

# Grid search on SVM
classifier = SVC()
grid = GridSearchCV(estimator=classifier, param_grid=param_grid,scoring='balanced_accuracy',refit = True,n_jobs = -1,verbose=2,cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 

print("Best parameters:", grid_results.best_params_)

#model with best parameters
model = grid_results.best_estimator_ 
print(model)



Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best parameters: {'C': 100, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf', 'random_state': 42}
SVC(C=100, class_weight='balanced', gamma='auto', random_state=42)
Wall time: 1h 2min 29s


In [ ]:
# from joblib import load
# grid_results= load(model_path+'SVMmodel.joblib') 

In [180]:
#check grid search results
results_df = pd.DataFrame(grid_results.cv_results_)
results_df= results_df.drop(["mean_fit_time","std_fit_time","mean_score_time","std_score_time","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score"],axis=1)
results_df

,param_C,param_class_weight,param_gamma,param_kernel,param_random_state,params,mean_test_score,std_test_score,rank_test_score
0,0.1,None,scale,linear,42,"{'C': 0.1, 'class_weight': None, 'gamma': 'sca...",0.500000,0.000000,58
1,0.1,None,scale,poly,42,"{'C': 0.1, 'class_weight': None, 'gamma': 'sca...",0.574882,0.013120,49
2,0.1,None,scale,rbf,42,"{'C': 0.1, 'class_weight': None, 'gamma': 'sca...",0.500000,0.000000,58
3,0.1,None,scale,sigmoid,42,"{'C': 0.1, 'class_weight': None, 'gamma': 'sca...",0.499584,0.000973,71
4,0.1,None,auto,linear,42,"{'C': 0.1, 'class_weight': None, 'gamma': 'aut...",0.500000,0.000000,58
...,...,...,...,...,...,...,...,...,...
75,1000,balanced,scale,sigmoid,42,"{'C': 1000, 'class_weight': 'balanced', 'gamma...",0.313386,0.013978,79
76,1000,balanced,auto,linear,42,"{'C': 1000, 'class_weight': 'balanced', 'gamma...",0.794029,0.011877,12
77,1000,balanced,auto,poly,42,"{'C': 1000, 'class_weight': 'balanced', 'gamma...",0.798166,0.009844,2
78,1000,balanced,auto,rbf,42,"{'C': 1000, 'class_weight': 'balanced', 'gamma...",0.797465,0.003781,5


In [181]:
import joblib
from sklearn import metrics

#make folder for SVM if it doesn't exist
import os
model_path = path + 'ModelBuilding/SVM/SNR_R5_ww51.4/' 
if not os.path.exists(model_path):
    os.makedirs(model_path)

#MODEL SAVING   
joblib.dump(model, model_path+"SVMmodel.joblib")

# Predicting the classes for validation set
y_pred = model.predict(X_val_scaled)

#overall accuracy
overall_acc = round(metrics.accuracy_score(y_val, y_pred)*100,2)
overall_acc = {'Overall Acc %':overall_acc}
overall_acc = pd.DataFrame([overall_acc])
overall_acc.to_csv(model_path+"Overall_Accuracy.csv")

#classification report
report = metrics.classification_report(y_val, y_pred,zero_division=0,output_dict=True)
report = pd.DataFrame(report).transpose()
report.to_csv(model_path+"Classification_Report.csv")

#confusion matrix with accuracies for each label
class_accuracies = []

for class_ in y_val.sort_values(ascending= True).unique():
    class_acc = round(np.mean(y_pred[y_val == class_] == class_)*100,2)
    class_accuracies.append(class_acc)
    
class_acc = pd.DataFrame(class_accuracies,index=['true:0', 'true:1'],columns= ["Accuracy %"])

cf_matrix = pd.DataFrame(
    metrics.confusion_matrix(y_val, y_pred, labels= [0, 1]), 
    index=['true:0', 'true:1'], 
    columns=['pred:0', 'pred:1']
)

ascend = None #input None/True/False to order the confusion matrix
if ascend == None:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1)
else:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1).sort_values(by=['Accuracy %'], ascending=ascend)

cf_matrix.to_csv(model_path+"Confusion_Matrix.csv")   

#validation results 
val_results = pd.concat([X_val_sf,X_val_scaled,pd.DataFrame(y_val),pd.DataFrame(y_pred,columns = ["PRED_SPECKLE"])],axis=1)
val_results.to_csv(model_path+"Val_results.csv",index=False) 


In [182]:
display(cf_matrix)
display(overall_acc)
display(report)

,pred:0,pred:1,Accuracy %
true:0,3317,886,78.92
true:1,88,449,83.61


,Overall Acc %
0,79.45


,precision,recall,f1-score,support
0,0.974156,0.789198,0.871977,4203.000000
1,0.336330,0.836127,0.479701,537.000000
accuracy,0.794515,0.794515,0.794515,0.794515
macro avg,0.655243,0.812662,0.675839,4740.000000
weighted avg,0.901896,0.794515,0.827535,4740.000000


### b) Random Forest


In [ ]:
X_train_scaled = pd.read_csv(dataprep_path+"TrainData_Corr0.95Rem.csv")
y_train = X_train_scaled["SPECKLE"]
X_train_scaled = X_train_scaled.drop(["SPECKLE"],axis=1)

X_val_scaled = pd.read_csv(dataprep_path+"ValData_Normalized.csv")
y_val = X_val_scaled["SPECKLE"]
X_val_scaled = X_val_scaled.drop(["SPECKLE"],axis=1)

#take top n important features only in train and validation
n=200
ns_df = pd.read_csv(featuresel_path+"Anova_Fs.csv")
features = ns_df.head(n)[["Feature"]] #choose the top n important features
X_train_scaled = X_train_scaled[features["Feature"]] #filter X_train_scaled with top n important features
X_val_scaled = X_val_scaled[features["Feature"]] #filter X_val_scaled with top n important features

print(X_train_scaled.shape)
print(X_val_scaled.shape)
print(y_train.value_counts())
print(y_val.value_counts())

In [ ]:
import time

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 300], # number of trees, default is 100
    'max_features': ['auto','log2'], #number of features to consider when looking for the best split, default auto
    'min_samples_leaf': [1,100], # minimum sample leaf (decision tree end node) size, default is 1
    'class_weight': [None,'balanced'], #weights for unbalanced data     
    'random_state': [42]
}

# Grid search on random forest
classifier = RandomForestClassifier() 
grid = GridSearchCV(estimator=classifier, param_grid=param_grid,scoring='balanced_accuracy',n_jobs = -1,verbose=2,cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 

print("Best parameters:", grid_results.best_params_)

#model with best parameters
model = grid_results.best_estimator_ 
print(model)

# fs_importance= pd.DataFrame(list(zip(X_train_scaled.columns, model.feature_importances_)), columns=['Feature', 'Importance']).sort_values(by=['Importance'], ascending=False)  
# display(fs_importance)

In [ ]:
#from joblib import load
#grid_results= load(path+'RFmodel.joblib') 

In [ ]:
#check grid search results
results_df = pd.DataFrame(grid_results.cv_results_)
results_df= results_df.drop(["mean_fit_time","std_fit_time","mean_score_time","std_score_time","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score"],axis=1)
results_df

In [ ]:
import joblib
from sklearn import metrics

#make folder for Random Forest if it doesn't exist
import os
model_path = path + 'ModelBuilding/RF/' 
if not os.path.exists(model_path):
    os.makedirs(model_path)

#MODEL SAVING   
joblib.dump(model, model_path+"RFmodel.joblib")

# Predicting the classes for validation set
y_pred = model.predict(X_val_scaled)

#Feature importance
fs_importance= pd.DataFrame(list(zip(X_train_scaled.columns, model.feature_importances_)), columns=['Feature', 'Importance']).sort_values(by=['Importance'], ascending=False)  
fs_importance.to_csv(model_path+"Feature_importance.csv",index=False) 

#overall accuracy
overall_acc = round(metrics.accuracy_score(y_val, y_pred)*100,2)
overall_acc = {'Overall Acc %':overall_acc}
overall_acc = pd.DataFrame([overall_acc])
overall_acc.to_csv(model_path+"Overall_Accuracy.csv")

#classification report
report = metrics.classification_report(y_val, y_pred,zero_division=0,output_dict=True)
report = pd.DataFrame(report).transpose()
report.to_csv(model_path+"Classification_Report.csv")

#confusion matrix with accuracies for each label
class_accuracies = []

for class_ in y_val.sort_values(ascending= True).unique():
    class_acc = round(np.mean(y_pred[y_val == class_] == class_)*100,2)
    class_accuracies.append(class_acc)
    
class_acc = pd.DataFrame(class_accuracies,index=['true:0', 'true:1'],columns= ["Accuracy %"])

cf_matrix = pd.DataFrame(
    metrics.confusion_matrix(y_val, y_pred, labels= [0, 1]), 
    index=['true:0', 'true:1'], 
    columns=['pred:0', 'pred:1']
)

ascend = None #input None/True/False to order the confusion matrix
if ascend == None:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1)
else:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1).sort_values(by=['Accuracy %'], ascending=ascend)

cf_matrix.to_csv(model_path+"Confusion_Matrix.csv")   

#validation results 
val_results = pd.concat([X_val_sf,X_val_scaled,pd.DataFrame(y_val),pd.DataFrame(y_pred,columns = ["PRED_SPECKLE"])],axis=1)
val_results.to_csv(model_path+"Val_results.csv",index=False) 


In [ ]:
y_val.value_counts()

In [ ]:
display(cf_matrix)
display(overall_acc)
display(report)

### c) XGBoost


In [5]:
X_train_scaled = pd.read_csv(dataprep_path+"TrainData_Corr0.95Rem.csv")
y_train = X_train_scaled["SPECKLE"]
X_train_scaled = X_train_scaled.drop(["SPECKLE"],axis=1)

X_val_scaled = pd.read_csv(dataprep_path+"ValData_Normalized.csv")
y_val = X_val_scaled["SPECKLE"]
X_val_scaled = X_val_scaled.drop(["SPECKLE"],axis=1)

#take top n important features only in train and validation
n=200
ns_df = pd.read_csv(featuresel_path+"Anova_Fs.csv")
features = ns_df.head(n)[["Feature"]] #choose the top n important features
X_train_scaled = X_train_scaled[features["Feature"]] #filter X_train_scaled with top n important features
X_val_scaled = X_val_scaled[features["Feature"]] #filter X_val_scaled with top n important features

print(X_train_scaled.shape)
print(X_val_scaled.shape)
print(y_train.value_counts())
print(y_val.value_counts())

(11057, 200)
(4740, 200)
0    9803
1    1254
Name: SPECKLE, dtype: int64
0    4203
1     537
Name: SPECKLE, dtype: int64


In [6]:
%%time
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 300], #number of trees, default=100
    'learning_rate': [0.01,0.1], # step size shrinkage used in update to prevent overfitting, default=0.3
    'min_child_weight': [1, 10], #used to control over-fitting,default=1
    'gamma': [0, 5], #specifies the minimum loss reduction required to make a split, default=0
    'subsample': [0.5, 1], #fraction of observations to be randomly sampled for each tree, default=1
    'colsample_bytree': [0.5, 1], #fraction of columns to be subsampled, default=1
    'max_depth': [3, 5], #size of decision trees, default=3
    'objective': ['binary:logistic'],    
    'scale_pos_weight': [1,5,9], #for class imbalance,default=1 (can be amount of majority/amount of minority in train)
    'random_state': [42]
}

# Grid search on XGBoost
classifier = XGBClassifier(use_label_encoder = False)
grid = GridSearchCV(estimator=classifier, param_grid=param_grid,scoring='balanced_accuracy',n_jobs = -1,verbose=2,cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 

print("Best parameters:", grid_results.best_params_)

#model with best parameters
model = grid_results.best_estimator_ 
print(model)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits
[17:49:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best parameters: {'colsample_bytree': 0.5, 'gamma': 5, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 10, 'n_estimators': 100, 'objective': 'binary:logistic', 'random_state': 42, 'scale_pos_weight': 9, 'subsample': 1}
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, gamma=5, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=10, missing=nan, monotone_constraints='()',
         

In [ ]:
# %%time
# from xgboost import XGBClassifier
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'n_estimators': [100, 600],
#     'learning_rate': [0.01],
#     'min_child_weight': [1, 10],
#     'gamma': [0.5, 5],
#     'subsample': [0.5, 1.0],
#     'colsample_bytree': [0.5, 1.0],
#     'max_depth': [3, 5],
#     'objective': ['binary:logistic'],
#     'nthread': [1],
#     'verbosity': [0],
#     'random_state': [42]
# }

# # Grid search on XGBoost
# classifier = XGBClassifier()
# grid = GridSearchCV(estimator=classifier, param_grid=param_grid, cv = 5)
# grid_results = grid.fit(X_train_scaled,y_train) 

# print("Best parameters:", grid_results.best_params_)

# #model with best parameters
# model = grid_results.best_estimator_ 
# print(model)

# # fs_importance= pd.DataFrame(list(zip(X_train_scaled.columns, model.feature_importances_)), columns=['Feature', 'Importance']).sort_values(by=['Importance'], ascending=False)  
# # display(fs_importance)

In [ ]:
# from joblib import load
# grid_results= load(model_path+'XGBmodel.joblib') 

In [7]:
#check grid search results
results_df = pd.DataFrame(grid_results.cv_results_)
results_df= results_df.drop(["mean_fit_time","std_fit_time","mean_score_time","std_score_time","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score"],axis=1)
results_df

,param_colsample_bytree,param_gamma,param_learning_rate,param_max_depth,param_min_child_weight,param_n_estimators,param_objective,param_random_state,param_scale_pos_weight,param_subsample,params,mean_test_score,std_test_score,rank_test_score
0,0.5,0,0.01,3,1,100,binary:logistic,42,1,0.5,"{'colsample_bytree': 0.5, 'gamma': 0, 'learnin...",0.533244,0.004548,377
1,0.5,0,0.01,3,1,100,binary:logistic,42,1,1,"{'colsample_bytree': 0.5, 'gamma': 0, 'learnin...",0.527182,0.003670,383
2,0.5,0,0.01,3,1,100,binary:logistic,42,5,0.5,"{'colsample_bytree': 0.5, 'gamma': 0, 'learnin...",0.747472,0.008482,183
3,0.5,0,0.01,3,1,100,binary:logistic,42,5,1,"{'colsample_bytree': 0.5, 'gamma': 0, 'learnin...",0.741111,0.008529,212
4,0.5,0,0.01,3,1,100,binary:logistic,42,9,0.5,"{'colsample_bytree': 0.5, 'gamma': 0, 'learnin...",0.764221,0.011145,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,1,5,0.1,5,10,300,binary:logistic,42,1,1,"{'colsample_bytree': 1, 'gamma': 5, 'learning_...",0.618699,0.013586,290
380,1,5,0.1,5,10,300,binary:logistic,42,5,0.5,"{'colsample_bytree': 1, 'gamma': 5, 'learning_...",0.698318,0.020064,245
381,1,5,0.1,5,10,300,binary:logistic,42,5,1,"{'colsample_bytree': 1, 'gamma': 5, 'learning_...",0.711755,0.015093,235
382,1,5,0.1,5,10,300,binary:logistic,42,9,0.5,"{'colsample_bytree': 1, 'gamma': 5, 'learning_...",0.721010,0.010667,229


In [12]:
X_val_sf = pd.read_csv(dataprep_path+"SupFeatures_Val.csv")

In [13]:
import joblib
import numpy as np
from sklearn import metrics

#make folder for XGBoost if it doesn't exist
import os
model_path = path + 'ModelBuilding/XGB/SNR_R5_ww51.4/' 
if not os.path.exists(model_path):
    os.makedirs(model_path)

#MODEL SAVING   
joblib.dump(model, model_path+"XGBmodel.joblib")

# Predicting the classes for validation set
y_pred = model.predict(X_val_scaled)

#Feature importance
fs_importance= pd.DataFrame(list(zip(X_train_scaled.columns, model.feature_importances_)), columns=['Feature', 'Importance']).sort_values(by=['Importance'], ascending=False)  
fs_importance.to_csv(model_path+"Feature_importance.csv",index=False) 

#overall accuracy
overall_acc = round(metrics.accuracy_score(y_val, y_pred)*100,2)
overall_acc = {'Overall Acc %':overall_acc}
overall_acc = pd.DataFrame([overall_acc])
overall_acc.to_csv(model_path+"Overall_Accuracy.csv")

#classification report
report = metrics.classification_report(y_val, y_pred,zero_division=0,output_dict=True)
report = pd.DataFrame(report).transpose()
report.to_csv(model_path+"Classification_Report.csv")

#confusion matrix with accuracies for each label
class_accuracies = []

for class_ in y_val.sort_values(ascending= True).unique():
    class_acc = round(np.mean(y_pred[y_val == class_] == class_)*100,2)
    class_accuracies.append(class_acc)
    
class_acc = pd.DataFrame(class_accuracies,index=['true:0', 'true:1'],columns= ["Accuracy %"])

cf_matrix = pd.DataFrame(
    metrics.confusion_matrix(y_val, y_pred, labels= [0, 1]), 
    index=['true:0', 'true:1'], 
    columns=['pred:0', 'pred:1']
)

ascend = None #input None/True/False to order the confusion matrix
if ascend == None:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1)
else:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1).sort_values(by=['Accuracy %'], ascending=ascend)

cf_matrix.to_csv(model_path+"Confusion_Matrix.csv")   

#validation results 
val_results = pd.concat([X_val_sf,X_val_scaled,pd.DataFrame(y_val),pd.DataFrame(y_pred,columns = ["PRED_SPECKLE"])],axis=1)
val_results.to_csv(model_path+"Val_results.csv",index=False) 


In [14]:
display(cf_matrix)
display(overall_acc)
display(report)

,pred:0,pred:1,Accuracy %
true:0,3256,947,77.47
true:1,100,437,81.38


,Overall Acc %
0,77.91


,precision,recall,f1-score,support
0,0.970203,0.774685,0.861490,4203.000000
1,0.315751,0.813780,0.454971,537.000000
accuracy,0.779114,0.779114,0.779114,0.779114
macro avg,0.642977,0.794233,0.658230,4740.000000
weighted avg,0.896059,0.779114,0.815435,4740.000000


### d) Deep Learning - MLP

In [ ]:
%%time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'hidden_layer_sizes': [(50,50)],
    'activation': ['logistic','tanh'],#['identity', 'logistic', 'tanh', 'relu']
    'solver': ['adam'], #['lbfgs', 'sgd', 'adam']
#     'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'random_state': [42]
}

# Grid search on MLP
classifier = MLPClassifier()
grid = GridSearchCV(estimator=classifier, param_grid=param_grid,scoring='balanced_accuracy',refit = True,n_jobs = -1,verbose=2,cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 

print("Best parameters:", grid_results.best_params_)

#model with best parameters
model = grid_results.best_estimator_ 
print(model)


In [ ]:
#check grid search results
results_df = pd.DataFrame(grid_results.cv_results_)
results_df= results_df.drop(["mean_fit_time","std_fit_time","mean_score_time","std_score_time","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score"],axis=1)
results_df

In [ ]:
import joblib
from sklearn import metrics

#make folder for MLP if it doesn't exist
import os
model_path = path + 'ModelBuilding/MLP/' 
if not os.path.exists(model_path):
    os.makedirs(model_path)

#MODEL SAVING   
joblib.dump(model, model_path+"MLPmodel.joblib")

# Predicting the classes for validation set
y_pred = model.predict(X_val_scaled)

#overall accuracy
overall_acc = round(metrics.accuracy_score(y_val, y_pred)*100,2)
overall_acc = {'Overall Acc %':overall_acc}
overall_acc = pd.DataFrame([overall_acc])
overall_acc.to_csv(model_path+"Overall_Accuracy.csv")

#classification report
report = metrics.classification_report(y_val, y_pred,zero_division=0,output_dict=True)
report = pd.DataFrame(report).transpose()
report.to_csv(model_path+"Classification_Report.csv")

#confusion matrix with accuracies for each label
class_accuracies = []

for class_ in y_val.sort_values(ascending= True).unique():
    class_acc = round(np.mean(y_pred[y_val == class_] == class_)*100,2)
    class_accuracies.append(class_acc)
    
class_acc = pd.DataFrame(class_accuracies,index=['true:0', 'true:1'],columns= ["Accuracy %"])

cf_matrix = pd.DataFrame(
    metrics.confusion_matrix(y_val, y_pred, labels= [0, 1]), 
    index=['true:0', 'true:1'], 
    columns=['pred:0', 'pred:1']
)

ascend = None #input None/True/False to order the confusion matrix
if ascend == None:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1)
else:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1).sort_values(by=['Accuracy %'], ascending=ascend)

cf_matrix.to_csv(model_path+"Confusion_Matrix.csv",index=False)   

#validation results 
val_results = pd.concat([X_val_sf,X_val_scaled,pd.DataFrame(y_val),pd.DataFrame(y_pred,columns = ["PRED_SPECKLE"])],axis=1)
val_results.to_csv(model_path+"Val_results.csv",index=False) 


### e) Deep Learning - TabNet

In [ ]:
%%time
import pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.model_selection import GridSearchCV

param_grid = {
    'hidden_layer_sizes': [(50,50)],
    'activation': ['logistic','tanh'],#['identity', 'logistic', 'tanh', 'relu']
    'solver': ['adam'], #['lbfgs', 'sgd', 'adam']
#     'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'random_state': [42]
}

# Grid search on MLP
classifier = TabNetClassifier()
grid = GridSearchCV(estimator=classifier, param_grid=param_grid,scoring='balanced_accuracy',refit = True,n_jobs = -1,verbose=2,cv = 5)
grid_results = grid.fit(X_train_scaled,y_train) 

print("Best parameters:", grid_results.best_params_)

#model with best parameters
model = grid_results.best_estimator_ 
print(model)


In [ ]:
%%time
import pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.model_selection import GridSearchCV



In [ ]:
%%time
import pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.model_selection import GridSearchCV

param_grid = {
    'optimizer_fn': [torch.optim.Adam]
    'optimizer_params': [dict(lr=2e-2)],
    'scheduler_params': ['gamma': 0.95, 'step_size': 10],
    'scheduler_fn': [torch.optim.lr_scheduler.StepLR],
    'mask_type': ['entmax'], #['entmax', 'sparsemax']
    'random_state': [42]
}

# Grid search on TabNet
classifier = TabNetClassifier()
grid = GridSearchCV(estimator=classifier, param_grid=param_grid)
grid_results = grid.fit(X_train_scaled,y_train, eval_set=[(X_train_scaled, y_train), (X_val_scaled, y_val)], 
                    eval_name=['train', 'valid'], max_epochs=1000, patience=50, batch_size=1580, virtual_batch_size=790, 
                    num_workers=0, weights=1, drop_last=False)
print("Best parameters:", grid_results.best_params_)

#model with best parameters
model = grid_results.best_estimator_ 
print(model)

In [ ]:
import pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.metrics import roc_auc_score, accuracy_score

# define the model
classifier = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":10, # how to use learning rate scheduler
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax"
                      )

# fit the model 
classifier.fit(
    X_train_scaled,y_train,
    eval_set=[(X_train_scaled, y_train), (X_val_scaled, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['auc','accuracy'],
    max_epochs=1000 , patience=50,
    batch_size=256, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
)     



#Notes:
#batch_size=15797*0.1=1580
#virtual_batch_size=15797*0.05=790
#X_train, X_val, y_train, y_val

### e) Deep Learning - 1D CNN

In [ ]:
%%time

### LOAD PACKAGES 
from numpy.random import seed
from keras.layers.convolutional import Conv1D, MaxPooling1D
# from keras.optimizers import SGD
from keras.optimizers import gradient_descent_v2
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.utils import class_weight

 
### FIT A 1D CONVOLUTIONAL NEURAL NETWORK
seed(42) # set seed for reproducible results
model = Sequential() #create a Sequential model
#add layers
model.add(Conv1D(20, 4, input_shape=x_train_scaled.shape[1:3], activation = 'relu')) #1D CNN layer
model.add(MaxPooling1D(pool_size=2)) #pooling layer reduces the learned features to 1/4 their size, consolidating them to only the most essential elements
model.add(Flatten()) #the learned features are flattened to one long vector
model.add(Dense(units = 1, activation = 'sigmoid')) # pass through a fully connected layer,units is the size of the output from the dense layer
#Compile Keras Model
sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
model.compile(loss = 'binary_crossentropy', optimizer = sgd, metrics = ['accuracy'])
#Add class weights for class imbalance in Keras
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)    
#Fit on train data
model.fit(X_train_scaled,y_train, class_weight=class_weights,batch_size = 500, epochs = 100, verbose = 0)


In [ ]:
import joblib
from sklearn import metrics

#make folder for CNN if it doesn't exist
import os
model_path = path + 'ModelBuilding/CNN/' 
if not os.path.exists(model_path):
    os.makedirs(model_path)

#MODEL SAVING   
joblib.dump(model, model_path+"CNNmodel.joblib")

# Predicting the classes for validation set
y_pred = model.predict(X_val_scaled)

#overall accuracy
overall_acc = round(metrics.accuracy_score(y_val, y_pred)*100,2)
overall_acc = {'Overall Acc %':overall_acc}
overall_acc = pd.DataFrame([overall_acc])
overall_acc.to_csv(model_path+"Overall_Accuracy.csv")

#classification report
report = metrics.classification_report(y_val, y_pred,zero_division=0,output_dict=True)
report = pd.DataFrame(report).transpose()
report.to_csv(model_path+"Classification_Report.csv")

#confusion matrix with accuracies for each label
class_accuracies = []

for class_ in y_val.sort_values(ascending= True).unique():
    class_acc = round(np.mean(y_pred[y_val == class_] == class_)*100,2)
    class_accuracies.append(class_acc)
    
class_acc = pd.DataFrame(class_accuracies,index=['true:0', 'true:1'],columns= ["Accuracy %"])

cf_matrix = pd.DataFrame(
    metrics.confusion_matrix(y_val, y_pred, labels= [0, 1]), 
    index=['true:0', 'true:1'], 
    columns=['pred:0', 'pred:1']
)

ascend = None #input None/True/False to order the confusion matrix
if ascend == None:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1)
else:
    cf_matrix = pd.concat([cf_matrix,class_acc],axis=1).sort_values(by=['Accuracy %'], ascending=ascend)

cf_matrix.to_csv(model_path+"Confusion_Matrix.csv",index=False)   

#validation results 
val_results = pd.concat([X_val_sf,X_val_scaled,pd.DataFrame(y_val),pd.DataFrame(y_pred,columns = ["PRED_SPECKLE"])],axis=1)
val_results.to_csv(model_path+"Val_results.csv",index=False) 

 